In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from sklearn.metrics import confusion_matrix


import nba_api
from nba_api import stats
from nba_api.stats.static import players
from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import commonallplayers
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.endpoints import leaguedashplayerbiostats
from nba_api.stats.endpoints import leaguedashplayerclutch
from nba_api.stats.endpoints import leaguedashteamstats
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.endpoints import playoffpicture
from nba_api.stats.endpoints import scoreboardv2
from nba_api.stats.endpoints import defensehub


#Import module to create our training and test sets.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


# Machine Learning
import statsmodels.api as sm 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, roc_auc_score, auc



# Display options
pd.options.display.max_columns = None
%matplotlib inline
sns.set(rc={'figure.figsize':(11.7,8.27)}) # Sets the size of plots to be a little bigger.
sns.set_style("darkgrid") # Set style of plots
sns.set_context("notebook") # Set plots to format for Jupyter
pd.options.display.float_format = '{:,.5f}'.format #Set to display only 5 decimal places


In [2]:
%%javascript
$('<div id="toc"></div>').css({position: 'fixed', top: '120px', left: 0}).appendTo(document.body);
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js');

<IPython.core.display.Javascript object>

# nba_api Basic Functions

## Season Player Stats

In [6]:
player_stats = leaguedashplayerbiostats.LeagueDashPlayerBioStats()

player_stats = player_stats.league_dash_player_bio_stats.get_data_frame()

player_stats.head()

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,PLAYER_HEIGHT,PLAYER_HEIGHT_INCHES,PLAYER_WEIGHT,COLLEGE,COUNTRY,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,GP,PTS,REB,AST,NET_RATING,OREB_PCT,DREB_PCT,USG_PCT,TS_PCT,AST_PCT
0,203932,Aaron Gordon,1610612753,ORL,23.00000,6-9,81.00000,220,Arizona,USA,2014,1,4,45,710,342,152,-0.90000,0.05100,0.17200,0.21000,0.54300,0.15700
1,1628988,Aaron Holiday,1610612754,IND,22.00000,6-1,73.00000,185,California-Los Angeles,USA,2018,1,23,28,142,40,38,10.60000,0.00700,0.12600,0.21500,0.52800,0.18500
2,1627846,Abdel Nader,1610612760,OKC,25.00000,6-6,78.00000,225,Iowa State,Egypt,2016,2,58,30,116,49,8,-15.20000,0.01700,0.15400,0.16100,0.58800,0.04500
3,201143,Al Horford,1610612738,BOS,32.00000,6-10,82.00000,245,Florida,Dominican Republic,2007,1,3,38,463,242,137,5.80000,0.05800,0.15800,0.18000,0.59200,0.18300
4,202329,Al-Farouq Aminu,1610612757,POR,28.00000,6-9,81.00000,220,Wake Forest,USA,2010,1,8,50,474,408,58,6.40000,0.05200,0.20700,0.12800,0.56400,0.04900


## Individual Clutch Stats

In [7]:
clutch_stats = leaguedashplayerclutch.LeagueDashPlayerClutch()

clutch_stats = clutch_stats.league_dash_player_clutch.get_data_frame()

clutch_stats.head()

,GROUP_SET,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,NBA_FANTASY_PTS,DD2,TD3,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,CFID,CFPARAMS
0,Players,203932,Aaron Gordon,1610612753,ORL,23.00000,25,13,12,0.52000,79.72500,10,22,0.45500,2,7,0.28600,2,4,0.50000,4,14,18,10,3,1,3,4,10,2,24,0,69.60000,10,0,19,24,350,158,50,65,65,131,84,84,127,153,130,182,44,29,31,21,67,95,20,368,352,166,87,157,56,23,15,1,"203932,1610612753"
1,Players,201143,Al Horford,1610612738,BOS,32.00000,22,13,9,0.59100,80.61833,14,21,0.66700,5,10,0.50000,12,14,0.85700,9,17,26,7,2,1,2,0,5,9,45,35,93.70000,3,0,42,24,283,103,49,43,72,28,35,61,31,36,46,83,13,16,9,42,88,95,44,1,254,50,44,20,28,55,15,1,"201143,1610612738"
2,Players,202329,Al-Farouq Aminu,1610612757,POR,28.00000,21,12,9,0.57100,72.33333,8,19,0.42100,6,16,0.37500,5,7,0.71400,6,18,24,2,1,2,1,1,5,6,27,11,66.80000,4,0,61,40,283,113,65,84,80,158,19,33,86,92,90,139,22,13,16,138,142,44,78,234,254,78,77,68,62,49,15,1,"202329,1610612757"
3,Players,202692,Alec Burks,1610612739,CLE,27.00000,10,5,5,0.50000,18.26167,4,11,0.36400,0,2,0.00000,5,6,0.83300,1,4,5,1,1,0,0,2,2,3,13,-5,19.50000,1,0,200,186,177,160,230,148,131,196,180,178,180,92,103,91,145,143,157,181,142,192,156,322,147,134,148,249,187,100,15,1,"202692,1610612739"
4,Players,203518,Alex Abrines,1610612760,OKC,25.00000,7,3,4,0.42900,11.86833,3,8,0.37500,2,6,0.33300,0,0,0.00000,0,1,1,0,0,0,1,0,1,0,8,4,12.20000,0,0,234,232,137,230,256,166,169,184,84,100,95,224,232,224,221,231,259,237,211,192,78,1,94,258,189,118,228,150,15,1,"203518,1610612760"


## Team Stats by Season

In [8]:
team_stats = leaguedashteamstats.LeagueDashTeamStats()

team_stats = team_stats.league_dash_team_stats.get_data_frame()

team_stats.head()

,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,CFID,CFPARAMS
0,1610612737,Atlanta Hawks,47,15,32,0.31900,"2,261.00000",1907,4228,0.45100,546,1620,0.33700,805,1087,0.74100,551,1599,2150,1204,882.00000,401,247,257,1117,1066,5165,-356.00000,25,26,26,26,28,23,24,20,14,9,27,22,18,24,6,24,20,14,30,11,17,20,29,9,22,26,10,Atlanta Hawks
1,1610612738,Boston Celtics,48,30,18,0.62500,"2,324.00000",1994,4326,0.46100,632,1709,0.37000,765,951,0.80400,488,1669,2157,1244,645.00000,424,266,188,999,954,5385,311.00000,17,7,7,7,21,16,12,14,2,4,6,24,29,4,18,16,19,7,4,8,10,2,9,27,18,3,10,Boston Celtics
2,1610612751,Brooklyn Nets,50,27,23,0.54000,"2,435.00000",2009,4414,0.45500,621,1721,0.36100,950,1265,0.75100,575,1735,2310,1170,769.00000,326,202,259,1103,1094,5589,20.00000,3,12,14,14,3,12,9,18,3,3,8,7,5,21,4,10,4,18,26,27,28,22,27,6,6,16,10,Brooklyn Nets
3,1610612766,Charlotte Hornets,48,23,25,0.47900,"2,329.00000",1962,4331,0.45300,564,1580,0.35700,890,1131,0.78700,476,1674,2150,1139,623.00000,342,239,320,935,1023,5378,25.00000,17,18,18,19,19,19,10,19,10,11,11,11,15,9,21,14,20,20,2,23,18,30,3,16,19,14,10,Charlotte Hornets
4,1610612741,Chicago Bulls,49,11,38,0.22400,"2,372.00000",1872,4231,0.44200,458,1312,0.34900,750,958,0.78300,399,1646,2045,1057,737.00000,388,216,282,1013,903,4952,-495.00000,7,27,28,27,8,24,23,27,28,26,16,26,28,10,29,18,26,27,23,12,25,27,12,30,29,29,10,Chicago Bulls


## Team Standings

In [20]:
# Eastern Conference Standings
EastConf = playoffpicture.PlayoffPicture()

EastConf = EastConf.east_conf_standings.get_data_frame()

,CONFERENCE,RANK,TEAM,TEAM_ID,WINS,LOSSES,PCT,DIV,CONF,HOME,AWAY,GB,GR_OVER_500,GR_OVER_500_HOME,GR_OVER_500_AWAY,GR_UNDER_500,GR_UNDER_500_HOME,GR_UNDER_500_AWAY,RANKING_CRITERIA,CLINCHED_PLAYOFFS,CLINCHED_CONFERENCE,CLINCHED_DIVISION,ELIMINATED_PLAYOFFS,SOSA_REMAINING
0,East,1,Milwaukee,1610612749,35,12,0.74500,8-1,23-7,22-4,13-8,0.00000,18,8,10,17,7,10,0,0,0,0,0,0.49480
1,East,2,Toronto,1610612761,36,15,0.70600,5-4,22-9,21-4,15-11,1.00000,12,9,3,19,7,12,0,0,0,0,0,0.45210
2,East,3,Indiana,1610612754,32,15,0.68100,7-2,24-8,18-6,14-9,3.00000,16,8,8,19,9,10,0,0,0,0,0,0.52200
3,East,4,Philadelphia,1610612755,32,17,0.65300,5-6,20-13,21-5,11-12,4.00000,18,11,7,15,4,11,0,0,0,0,0,0.51240
4,East,5,Boston,1610612738,30,18,0.62500,7-3,21-10,19-5,11-13,5.50000,21,11,10,13,6,7,0,0,0,0,0,0.50930
5,East,6,Brooklyn,1610612751,27,23,0.54000,7-4,19-12,15-11,12-12,9.50000,19,7,12,13,8,5,0,0,0,0,0,0.51350
6,East,7,Miami,1610612748,23,24,0.48900,3-8,13-17,11-13,12-11,12.00000,20,8,12,15,9,6,0,0,0,0,0,0.53670
7,East,8,Charlotte,1610612766,23,25,0.47900,6-2,17-14,16-8,7-17,12.50000,18,9,9,16,8,8,0,0,0,0,0,0.52110
8,East,9,Washington,1610612764,21,27,0.43800,6-4,14-17,15-9,6-18,14.50000,16,8,8,18,9,9,5,0,0,0,0,0.48010
9,East,10,Detroit,1610612765,21,27,0.43800,4-4,14-17,13-11,8-16,14.50000,17,9,8,17,8,9,0,0,0,0,0,0.45570


In [21]:
# Western Conference Standings
WestConf = playoffpicture.PlayoffPicture()

WestConf = WestConf.west_conf_standings.get_data_frame()

In [25]:
# Append into one df
Standings = EastConf.append(WestConf).reindex()

In [26]:
Standings

,CONFERENCE,RANK,TEAM,TEAM_ID,WINS,LOSSES,PCT,DIV,CONF,HOME,AWAY,GB,GR_OVER_500,GR_OVER_500_HOME,GR_OVER_500_AWAY,GR_UNDER_500,GR_UNDER_500_HOME,GR_UNDER_500_AWAY,RANKING_CRITERIA,CLINCHED_PLAYOFFS,CLINCHED_CONFERENCE,CLINCHED_DIVISION,ELIMINATED_PLAYOFFS,SOSA_REMAINING
0,East,1,Milwaukee,1610612749,35,12,0.74500,8-1,23-7,22-4,13-8,0.00000,18,8,10,17,7,10,0,0,0,0,0,0.49480
1,East,2,Toronto,1610612761,36,15,0.70600,5-4,22-9,21-4,15-11,1.00000,12,9,3,19,7,12,0,0,0,0,0,0.45210
2,East,3,Indiana,1610612754,32,15,0.68100,7-2,24-8,18-6,14-9,3.00000,16,8,8,19,9,10,0,0,0,0,0,0.52200
3,East,4,Philadelphia,1610612755,32,17,0.65300,5-6,20-13,21-5,11-12,4.00000,18,11,7,15,4,11,0,0,0,0,0,0.51240
4,East,5,Boston,1610612738,30,18,0.62500,7-3,21-10,19-5,11-13,5.50000,21,11,10,13,6,7,0,0,0,0,0,0.50930
5,East,6,Brooklyn,1610612751,27,23,0.54000,7-4,19-12,15-11,12-12,9.50000,19,7,12,13,8,5,0,0,0,0,0,0.51350
6,East,7,Miami,1610612748,23,24,0.48900,3-8,13-17,11-13,12-11,12.00000,20,8,12,15,9,6,0,0,0,0,0,0.53670
7,East,8,Charlotte,1610612766,23,25,0.47900,6-2,17-14,16-8,7-17,12.50000,18,9,9,16,8,8,0,0,0,0,0,0.52110
8,East,9,Washington,1610612764,21,27,0.43800,6-4,14-17,15-9,6-18,14.50000,16,8,8,18,9,9,5,0,0,0,0,0.48010
9,East,10,Detroit,1610612765,21,27,0.43800,4-4,14-17,13-11,8-16,14.50000,17,9,8,17,8,9,0,0,0,0,0,0.45570


## Game Logs

In [ ]:
game_log = leaguegamelog.LeagueGameLog(date_from_nullable='1-1-2019')

game_log = game_log.league_game_log.get_data_frame()

game_log.head()

## Player Game Logs

In [ ]:
player_game_log = playergamelog.PlayerGameLog(player_id = 2546,season_all='ALL')

player_game_log = player_game_log.player_game_log.get_data_frame()

player_game_log.head()

## Individual Stat Team Leaders

In [ ]:
## Get Individual Stat Team Leaders

team_ind_lead = scoreboardv2.ScoreboardV2()

team_ind_lead = team_ind_lead.team_leaders.get_data_frame()

team_ind_lead.head()

## Real Time Line Scores

In [ ]:
# These stats are up to the moment
line_score = scoreboardv2.ScoreboardV2()

line_score = line_score.line_score.get_data_frame()

line_score.head()

# Previous 2 Years Stats

## Player Season Stats

In [ ]:
clutch_stats_1516 = leaguedashplayerclutch.LeagueDashPlayerClutch(season='2015-16')

clutch_stats_1516 = clutch_stats_1516.league_dash_player_clutch.get_data_frame()

clutch_stats_1516.head()

In [ ]:
player_stats_1516 = leaguedashplayerbiostats.LeagueDashPlayerBioStats(season='2015-16')

player_stats_1516 = player_stats_1516.league_dash_player_bio_stats.get_data_frame()

player_stats_1516.head()

In [ ]:
clutch_stats_1617 = leaguedashplayerclutch.LeagueDashPlayerClutch(season='2016-17')

clutch_stats_1617 = clutch_stats_1617.league_dash_player_clutch.get_data_frame()

clutch_stats_1617.head()

In [ ]:
player_stats_1617 = leaguedashplayerbiostats.LeagueDashPlayerBioStats(season='2016-17')

player_stats_1617 = player_stats_1617.league_dash_player_bio_stats.get_data_frame()

player_stats_1617.head()

## Team Season Stats

In [ ]:
team_stats_1516 = leaguedashteamstats.LeagueDashTeamStats(season='2015-16')

team_stats_1516 = team_stats_1516.league_dash_team_stats.get_data_frame()

team_stats_1516.head()

In [ ]:
team_stats_1617 = leaguedashteamstats.LeagueDashTeamStats(season='2016-17')

team_stats_1617 = team_stats_1617.league_dash_team_stats.get_data_frame()

team_stats_1617.head()

# Get Player and Team Lists

In [ ]:
from nba_api.stats.static import players

# get all players
players = players.get_players()

# put in df
players = pd.DataFrame(players)

players.head()

In [ ]:
from nba_api.stats.static import teams

# Get all teams.
teams = teams.get_teams()

# put in df
teams = pd.DataFrame(teams)

teams.head()

## Get All Current Players

In [ ]:
all_players = commonallplayers.CommonAllPlayers(is_only_current_season=1)

all_players = all_players.common_all_players.get_data_frame()

all_players

## Current Team Rosters

In [ ]:
rosters = commonteamroster.CommonTeamRoster(team_id=1610612738)

rosters = rosters.common_team_roster.get_data_frame()

rosters